In [1]:
!pip3 install pymongo

In [2]:
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
#30% of the company staff have at least 1 child.
#Developers like to be near successful tech startups that have raised at least 1 Million dollars.
#Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
#Account managers need to travel a lot.
#Everyone in the company is between 25 and 40, give them some place to go party.
#The CEO is vegan.
#If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
#The office dog—"Pepe" needs a hairdresser every month. Ensure there's one not too far away.

In [151]:
#importing necessary libraries for geo_visualization
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv
from pymongo import MongoClient

In [4]:
conn = MongoClient("localhost:27017")

In [5]:
# choose database to work with
db = conn.get_database("ironhack")

# choose collection to work with
collection = db.get_collection("companies")

In [6]:
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [7]:
c = collection

In [8]:
data_category = c.find({"category_code": "games_video"})

In [9]:
category_gaming = list(data_category)

In [10]:
len(category_gaming)

1083

In [11]:
data_dublin = c.find({"offices.city": "Dublin"})

In [12]:
location_dublin = list(data_dublin)

In [13]:
len(location_dublin)

66

In [14]:
#collection.find_one({"offices.city": "Dublin"})

In [15]:
pd.DataFrame(c.find(
    {"$and": [
        {"offices.city": 'Dublin'},
        {"total_money_raised": {'$gt': '1.0'}}
    ]},
    {"name": 1, "rating": 1, "total_money_raised": 1, 'offices.latitude':1, 'offices.longitude':1}
))

,_id,name,total_money_raised,offices
0,52cdef7c4bab8bd6752986b7,eRepublik,€2.75M,"[{'latitude': 40.43094, 'longitude': -3.695289..."
1,52cdef7d4bab8bd675298963,Iceberg,€400k,"[{'latitude': 53.344104, 'longitude': -6.267494}]"
2,52cdef7d4bab8bd675298aec,Dial2Do,€1M,"[{'latitude': 53.34202, 'longitude': -6.250684}]"
3,52cdef7d4bab8bd675299e60,payByMobile,€750k,"[{'latitude': None, 'longitude': None}]"
4,52cdef7e4bab8bd67529b0e2,DediServe,€1M,"[{'latitude': None, 'longitude': None}, {'lati..."
5,52cdef7e4bab8bd67529b59e,Branded Payment Solutions,€100k,"[{'latitude': 53.344104, 'longitude': -6.26749..."
6,52cdef7e4bab8bd67529b913,Xiam,€1M,"[{'latitude': None, 'longitude': None}]"
7,52cdef7f4bab8bd67529c55b,Clavis Technology,€3.66M,"[{'latitude': None, 'longitude': None}, {'lati..."


In [16]:
#Dial2Do, eRepublik

In [17]:
#latitude and longitude can be found in the office heading too

In [18]:
load_dotenv()

True

In [19]:
donde = "Dublin"
def geocode(address):
    """
    Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [20]:
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()
data

{'standard': {'addresst': {},
  'city': 'Dublin',
  'prov': 'IE',
  'countryname': 'Ireland',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-6.25415',
 'alt': {'loc': {'longt': '-6.30149',
   'prov': 'IE',
   'city': 'Dublin',
   'countryname': 'Ireland',
   'postal': 'D11Y895',
   'region': {},
   'latt': '53.40560'}},
 'elevation': {},
 'latt': '53.32306'}

In [21]:
Dublin = geocode(donde)

In [22]:
#longitud - latitud
Dublin

{'type': 'Point', 'coordinates': [-6.25415, 53.32306]}

In [23]:
dublin = {'type': 'Point', 'coordinates': [-6.25415, 53.32306]}

In [24]:
initial_lat = 53.32306
initial_long = -6.25415

In [25]:
map_dublin = folium.Map(location = [53.32306,-6.25415], zoom_start = 15)
#map_dublin

In [26]:
#Adding Dial2Do
dial2do = Marker(location=[initial_lat,initial_long], tooltip = "Dial2Do")
dial2do.add_to(map_dublin)

In [27]:
#Location Dial2do
dial2do_lat = 53.34202
dial2do_long = -6.250684

In [28]:
#creating the icon for Dial2do
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Dial2do"
)
loc = [dial2do_lat, dial2do_long]
#saving the marker in one variable
marker_dial2do = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_dial2do.add_to(map_dublin)
#map_dublin

In [29]:
#Adding Iceberg (even though raising <€1.0M)
#Location Iceberg
iceberg_lat =  53.344104
iceberg_long = -6.267494

#creating the icon for Iceberg
icono = Icon(color = "green",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Iceberg"
)
loc = [iceberg_lat, iceberg_long]
#saving the marker in one variable
marker_iceberg = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_iceberg.add_to(map_dublin)

#map_dublin

In [30]:
#Adding Clavis Technologies (searched coordinates in google maps)
#Location Clavis
clavis_lat =  53.346976
clavis_long = -6.258292

#creating the icon for Clavis
icono = Icon(color = "red",
             prefix = "fa",
             icon = "briefcase",
             icon_color = "black",
             tooltip = "Clavis Tech"
)
loc = [clavis_lat, clavis_long]
#saving the marker in one variable
marker_clavis = Marker(location = loc, icon = icono)
#let's add the marker to the original dublin map where we already had Dial2do location saved
marker_clavis.add_to(map_dublin)

map_dublin

In [31]:
clavis_tech = {'type': 'Point', 'coordinates': [-6.258292, 53.346976]}

# Calling FourSquare API

### Starbucks

In [94]:
load_dotenv()

True

In [95]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [118]:
enlace = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [119]:
print(clavis_tech.get("coordinates")[0])

-6.258292


In [120]:
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{starbucks}",
          "limit" : 10
}

In [121]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [122]:
#data
data.keys()

dict_keys(['meta', 'response'])

In [123]:
decoding_data = data.get('response')

In [124]:
#for k,v in decoding_data.items(): print (k,v,"\n")

In [125]:
decoded = decoding_data.get('groups')[0]

In [126]:
#for k,v in decoded.items(): print(k,v,"\n")

In [127]:
starbucks = decoded.get('items')

In [128]:
starbucks[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bf46902e5eba5939ca21f90',
  'name': 'DoubleTree by Hilton',
  'contact': {},
  'location': {'address': 'Ormond Quay Lower',
   'lat': 53.34655895414743,
   'lng': -6.265900795222777,
   'labeledLatLngs': [{'label': 'display',
     'lat': 53.34655895414743,
     'lng': -6.265900795222777}],
   'distance': 507,
   'postalCode': '1',
   'cc': 'IE',
   'city': 'Dublin',
   'state': 'Dublin City',
   'country': 'Ireland',
   'formattedAddress': ['Ormond Quay Lower',
    'Dublin',
    'Dublin City',
    'Ireland']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
 

In [129]:
from functools import reduce
import operator

In [130]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [131]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [132]:
new_list = []
for diccionario in starbucks:
    starbuckslist = {}
    starbuckslist["name"] = getFromDict(diccionario,mapa_nombre)
    starbuckslist["latitud"] = getFromDict(diccionario,latitud)
    starbuckslist["longitud"] = getFromDict(diccionario,longitud)
    new_list.append(starbuckslist)

In [133]:
dataframestarbucks = pd.DataFrame(new_list)
dataframestarbucks.head()

,name,latitud,longitud
0,DoubleTree by Hilton,53.346559,-6.265901
1,Farrier & Draper,53.342417,-6.262255
2,"The Shelbourne Dublin, A Renaissance Hotel",53.338972,-6.256014
3,TOSS'D Noodles & Salads,53.348874,-6.247886
4,FXB Steak & Seafood,53.337314,-6.251954


In [134]:
import geopandas as gpd

In [135]:
gdf = gpd.GeoDataFrame(dataframestarbucks, geometry=gpd.points_from_xy(dataframestarbucks.longitud, dataframestarbucks.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,DoubleTree by Hilton,53.346559,-6.265901,POINT (-6.26590 53.34656)
1,Farrier & Draper,53.342417,-6.262255,POINT (-6.26226 53.34242)
2,"The Shelbourne Dublin, A Renaissance Hotel",53.338972,-6.256014,POINT (-6.25601 53.33897)
3,TOSS'D Noodles & Salads,53.348874,-6.247886,POINT (-6.24789 53.34887)
4,FXB Steak & Seafood,53.337314,-6.251954,POINT (-6.25195 53.33731)


In [136]:
from cartoframes.viz import Map, Layer, popup_element

In [137]:
Map(Layer(gdf, popup_hover=[popup_element("name","Starbucks")]))

### Irish Pubs

In [56]:
#Looking for some place to party - we will try focusing on the River Liffey surroundings
enlace = 'https://api.foursquare.com/v2/venues/explore'
irish_pubs = "52e81612bcbc57f1066b7a06"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{irish_pubs}",
          "limit" : 10
}

In [57]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [58]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
irish_pubs = decoded.get('items')

In [59]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [60]:
new_list2 = []
for diccionario in irish_pubs:
    irishpubslist = {}
    irishpubslist["name"] = getFromDict(diccionario,mapa_nombre)
    irishpubslist["latitud"] = getFromDict(diccionario,latitud)
    irishpubslist["longitud"] = getFromDict(diccionario,longitud)
    new_list2.append(irishpubslist)
    
dataframeirish = pd.DataFrame(new_list2)
#dataframeirish.head()

In [61]:
gdf = gpd.GeoDataFrame(dataframeirish, geometry=gpd.points_from_xy(dataframeirish.longitud, dataframeirish.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,Trinity College Old Library & The Book of Kell...,53.343692,-6.256907,POINT (-6.25691 53.34369)
1,FXB Grill,53.344593,-6.263811,POINT (-6.26381 53.34459)
2,Fallon & Byrne Food Store,53.343074,-6.263233,POINT (-6.26323 53.34307)
3,Fallon & Byrne Wine Cellar,53.343078,-6.263275,POINT (-6.26328 53.34308)
4,FXB Steak & Seafood,53.337314,-6.251954,POINT (-6.25195 53.33731)


In [62]:
Map(Layer(gdf, popup_hover=[popup_element("name","Irish Pubs")]))

### Veterinarian

In [63]:
#Looking for dog vet service
enlace = 'https://api.foursquare.com/v2/venues/explore'
vet = "4d954af4a243a5684765b473"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{vet}",
          "limit" : 10
}

In [64]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)

In [65]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
vet = decoded.get('items')

In [66]:
new_list3 = []
for diccionario in vet:
    vetlist = {}
    vetlist["name"] = getFromDict(diccionario,mapa_nombre)
    vetlist["latitud"] = getFromDict(diccionario,latitud)
    vetlist["longitud"] = getFromDict(diccionario,longitud)
    new_list3.append(vetlist)
    
dataframevet = pd.DataFrame(new_list3)
#dataframevet.head()

In [67]:
gdf = gpd.GeoDataFrame(dataframevet, geometry=gpd.points_from_xy(dataframevet.longitud, dataframevet.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,DoubleTree by Hilton,53.346559,-6.265901,POINT (-6.26590 53.34656)
1,4 Dame Lane,53.343803,-6.262848,POINT (-6.26285 53.34380)
2,"The Shelbourne Dublin, A Renaissance Hotel",53.338972,-6.256014,POINT (-6.25601 53.33897)
3,Dublin Bus Stop No 289,53.348976,-6.256274,POINT (-6.25627 53.34898)
4,Enterprise Rent-A-Car,53.359357,-6.254378,POINT (-6.25438 53.35936)


In [68]:
#https://www.w3resource.com/pandas/dataframe/dataframe-explode.php Primero el explode, y con el explode, sacar las columnas

In [69]:
#Looking for vegan rests
enlace = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{vegan}",
          "limit" : 10
}

In [70]:
resp = requests.get (url = enlace, params = parametros)
data = json.loads(resp.text)
data

{'meta': {'code': 200, 'requestId': '5fb2b6d5d859ad1d9ff1da23'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 1809,
  'headerLocation': 'South-East Inner City',
  'headerFullLocation': 'South-East Inner City, Dublin',
  'headerLocationGranularity': 'neighborhood',
  'query': '4bf58dd8d48988d1d3941735',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 53.35403576871546,
    'lng': -6.246463524122386},
   'sw': {'lat': 53.34027013758809, 'lng': -6.2777577945291485}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52dd4d27498ec2361514b507',
       'name': '147 Deli',
       'contact': {},
       'location': {'address': '147 Parnell St',
        'lat': 53.35341005820966,
 

In [71]:
decoding_data = data.get('response')
decoded = decoding_data.get('groups')[0]
restaurantes = decoded.get("items")

In [72]:
mapa_nombre = ["venue","name"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]

In [73]:
new_list4 = []
for diccionario in restaurantes:
    veglist = {}
    veglist["name"] = getFromDict(diccionario,mapa_nombre)
    veglist["latitud"] = getFromDict(diccionario,latitud)
    veglist["longitud"] = getFromDict(diccionario,longitud)
    new_list3.append(veglist)
    
dataframeveg = pd.DataFrame(new_list4)
dataframeveg.head()

""


In [74]:
#Looking for airport connections
enlace = 'https://api.foursquare.com/v2/venues/explore'
train_station = "4bf58dd8d48988d129951735"
parametros = { "client_id" : tok1,
          "client_secret" : tok2,
          "v" : "20180323",
          "ll": f"{clavis_tech.get('coordinates')[1]},{clavis_tech.get('coordinates')[0]}",
          "query": f"{train_station}",
          "limit" : 20
}

In [75]:
resp_tram = requests.get (url = enlace, params = parametros)
data_tram = json.loads(resp.text)
data_tram

{'meta': {'code': 200, 'requestId': '5fb2b6d5d859ad1d9ff1da23'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 1809,
  'headerLocation': 'South-East Inner City',
  'headerFullLocation': 'South-East Inner City, Dublin',
  'headerLocationGranularity': 'neighborhood',
  'query': '4bf58dd8d48988d1d3941735',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 53.35403576871546,
    'lng': -6.246463524122386},
   'sw': {'lat': 53.34027013758809, 'lng': -6.2777577945291485}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52dd4d27498ec2361514b507',
       'name': '147 Deli',
       'contact': {},
       'location': {'address': '147 Parnell St',
        'lat': 53.35341005820966,
 

In [76]:
decoding_data = data_tram.get('response')
decoded = decoding_data.get('groups')[0]
trams = decoded.get("items")

In [77]:
map_name = ["venue","name"]
lat = ["venue", "location","lat"]
long = ["venue","location","lng"]

In [78]:
new_list_tram = []
for diccionario in trams:
    airlist = {}
    airlist["name"] = getFromDict(diccionario,mapa_nombre)
    airlist["latitud"] = getFromDict(diccionario,latitud)
    airlist["longitud"] = getFromDict(diccionario,longitud)
    new_list_tram.append(airlist)
    
dataframe_trams = pd.DataFrame(new_list_tram)
dataframe_trams.head()

,name,latitud,longitud
0,147 Deli,53.353410,-6.259807
1,DoubleTree by Hilton,53.346559,-6.265901
2,4 Dame Lane,53.343803,-6.262848
3,TOSS'D Noodles & Salads,53.348874,-6.247886
4,District 8,53.342221,-6.276335


In [79]:
#Base map is the location of Clavis Tech
map_final = Map(location=[53.346976,-6.258292],zoom_start=15)

In [138]:
dataframestarbucks.head()

,name,latitud,longitud,geometry
0,DoubleTree by Hilton,53.346559,-6.265901,POINT (-6.26590 53.34656)
1,Farrier & Draper,53.342417,-6.262255,POINT (-6.26226 53.34242)
2,"The Shelbourne Dublin, A Renaissance Hotel",53.338972,-6.256014,POINT (-6.25601 53.33897)
3,TOSS'D Noodles & Salads,53.348874,-6.247886,POINT (-6.24789 53.34887)
4,FXB Steak & Seafood,53.337314,-6.251954,POINT (-6.25195 53.33731)


In [142]:
# Calculate Distance Between GPS Points

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [146]:
#Let's see which starbucks is closer to Clavis Tech
clavis_tech_coord = 53.346976,  -6.258292
starbucks = {
    'DoubleTree by Hilton': (53.346559,  -6.265901),
    'Farrier & Draper': (53.342417,  -6.262255),
    'The Shelbourne Dublin': (53.338972, -6.247886),
    'TOSSD Noodles & Salads': (53.348874,  -6.247886),
    'FXB Steak & Seafood': (53.337314,  -6.251954)
}

for places, coord in starbucks.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see DoubleTree Hilton is 500m away -> chosen starbucks

DoubleTree by Hilton 507.3533169653642
Farrier & Draper 571.2965056892294
The Shelbourne Dublin 1126.9636091398845
TOSSD Noodles & Salads 722.4592739043518
FXB Steak & Seafood 1154.1466690405266


In [148]:
dataframeirish.head()

,name,latitud,longitud,geometry
0,Trinity College Old Library & The Book of Kell...,53.343692,-6.256907,POINT (-6.25691 53.34369)
1,FXB Grill,53.344593,-6.263811,POINT (-6.26381 53.34459)
2,Fallon & Byrne Food Store,53.343074,-6.263233,POINT (-6.26323 53.34307)
3,Fallon & Byrne Wine Cellar,53.343078,-6.263275,POINT (-6.26328 53.34308)
4,FXB Steak & Seafood,53.337314,-6.251954,POINT (-6.25195 53.33731)


In [150]:
#Let's now calculate distantes to irish pubs
clavis_tech_coord = 53.346976,  -6.258292
irish_pubs = {
    'Trinity College': (53.343692,  -6.256907),
    'FXB Grill': (53.344593,  -6.263811),
    'Fallon & Byrne Food Store': (53.343074, -6.263233),
    'Fallon & Byrne Wine Cellar': (53.343078,  -6.263275),
    'FXB Steak & Seafood': (53.337314,  -6.251954)
}

for places, coord in irish_pubs.items():
    distance = haversine(clavis_tech_coord, coord)
    print(places, distance)
#We can see there are many irish pubs nearby: Trinity College and FXB Grill less than 500m away

Trinity College 376.66677437137986
FXB Grill 452.2702595799134
Fallon & Byrne Food Store 544.0621470953885
Fallon & Byrne Wine Cellar 545.3947205415539
FXB Steak & Seafood 1154.1466690405266
